In [ ]:
import yfinance as yf
import pandas as pd
import ta

# Definir o ticker da ação desejada
ticker = "AAPL"

# Baixar os dados da ação dos últimos 30 dias
dados = yf.download(ticker, period="30d", interval="1d")

# Selecionar apenas as colunas relevantes e garantir que sejam séries unidimensionais
dados = dados[['Close', 'High', 'Low', 'Open', 'Volume']].apply(pd.Series.squeeze)

# Calcular Médias Móveis
dados['SMA_10'] = dados['Close'].rolling(window=10).mean()
dados['SMA_20'] = dados['Close'].rolling(window=20).mean()

# Calcular o RSI (Relative Strength Index) de 14 dias
rsi_indicator = ta.momentum.RSIIndicator(close=dados["Close"].squeeze(), window=14)
dados["RSI"] = rsi_indicator.rsi()

# Calcular as Bandas de Bollinger (20 dias)
bollinger = ta.volatility.BollingerBands(dados["Close"], window=20)
dados["Bollinger_High"] = bollinger.bollinger_hband()
dados["Bollinger_Low"] = bollinger.bollinger_lband()

# Criar função de recomendação
def gerar_recomendacao(df):
    if len(df) < 20:  # Garantir que haja dados suficientes para análise
        return "Dados insuficientes para gerar recomendação."

    ult_linha = df.iloc[-1]  # Última linha dos dados
    prev_linha = df.iloc[-2]  # Linha anterior para verificar cruzamentos
    recomendacoes = []

    # Analisando RSI
    if ult_linha["RSI"] < 30:
        recomendacoes.append("RSI indica COMPRA (ativo sobrevendido).")
    elif ult_linha["RSI"] > 70:
        recomendacoes.append("RSI indica VENDA (ativo sobrecomprado).")

    # Cruzamento de Médias Móveis
    if prev_linha["SMA_10"] < prev_linha["SMA_20"] and ult_linha["SMA_10"] > ult_linha["SMA_20"]:
        recomendacoes.append("Cruzamento de Médias Móveis indica COMPRA.")
    elif prev_linha["SMA_10"] > prev_linha["SMA_20"] and ult_linha["SMA_10"] < ult_linha["SMA_20"]:
        recomendacoes.append("Cruzamento de Médias Móveis indica VENDA.")

    # Bandas de Bollinger
    if ult_linha["Close"] < ult_linha["Bollinger_Low"]:
        recomendacoes.append("Banda de Bollinger indica COMPRA (preço abaixo da banda inferior).")
    elif ult_linha["Close"] > ult_linha["Bollinger_High"]:
        recomendacoes.append("Banda de Bollinger indica VENDA (preço acima da banda superior).")

    # Se nenhuma recomendação clara for encontrada
    if not recomendacoes:
        return "Nenhuma recomendação clara no momento."
    
    return "\n".join(recomendacoes)

# Gerar e exibir recomendação
recomendacao = gerar_recomendacao(dados)
print("\nRecomendação baseada nos indicadores:")
print(recomendacao)
